In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

df_records_raw = catalog.load('raw/oai/records#parquet')

[12/06/25 11:19:38] INFO     Loading data from raw/oai/records#parquet (ParquetDataset)...     ]8;id=676922;file:///home/pablo/dev/scholar/kedro-scholar/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=736206;file:///home/pablo/dev/scholar/kedro-scholar/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#1048\1048]8;;\

In [2]:
df_records_raw

,record_id,datestamp,set_id,col_id,title,date_issued,creators,types,identifiers,languages,subjects,publishers,relations,rights,extract_datetime
0,oai:ri.conicet.gov.ar:11336/179477,2024-01-12T04:51:20Z,"[com_11336_35, com_11336_14, col_11336_36, snrd]",com_11336_35,Paisajes mesetarios en Patagonia: Tecnología d...,2016-12,"[Cassiodoro, Gisela Eva]","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/179477, Cassiodor...",[spa],"[MESETAS, TECNOLOGÍA, USO DEL ESPACIO, LOGÍSTI...",[Instituto Nacional de Antropología y Pensamie...,[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr...",2025-12-06 00:00:00+00:00
1,oai:ri.conicet.gov.ar:11336/116972,2020-10-27T20:15:28Z,"[com_11336_219, com_11336_218, col_11336_89919...",com_11336_219,Distancias entre la ecología y la praxis ambie...,2011,"[Nuñez, Paula Gabriela]","[info:eu-repo/semantics/publishedVersion, info...","[http://hdl.handle.net/11336/116972, Nuñez, Pa...",[spa],"[ECOLOGÍA, PRAXIS AMBIENTAL, ECOFEMINISMO, TEO...",[Universidad Nacional de La Plata],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr...",2025-12-06 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/144165,2021-10-18T20:30:01Z,"[com_11336_238, com_11336_231, col_11336_239, ...",com_11336_238,Simultaneous Determination of Human Erythrocyt...,2020-10,"[Londero, Carolina María, Riquelme, Bibiana Do...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/144165, Londero, ...",[eng],"[HEMORHEOLOGY, MICROFLUIDIC CHAMBER, RED BLOOD...",[Humana Press],[info:eu-repo/semantics/altIdentifier/doi/10.1...,"[info:eu-repo/semantics/restrictedAccess, http...",2025-12-06 00:00:00+00:00
197,oai:ri.conicet.gov.ar:11336/54764,2023-02-16T13:22:10Z,"[com_11336_345, com_11336_332, col_11336_346, ...",com_11336_345,Two-dimensional geoelectrical modeling using a...,2000-05,"[Osella, Ana Maria, Martinelli, Hilda Patricia...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/54764, Osella, An...",[eng],"[Dc Resistivity Method, Geoelectrical Prospect...",[Institute of Electrical and Electronics Engin...,[info:eu-repo/semantics/altIdentifier/doi/10.1...,"[info:eu-repo/semantics/openAccess, https://cr...",2025-12-06 00:00:00+00:00


## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [6]:
def oai_load_records(df_records_raw: pd.DataFrame)-> pd.DataFrame:

    df_records = df_records_raw[['record_id','col_id','title','date_issued', 'extract_datetime']]

    df_record_creators = df_records_raw[['record_id','creators', 'extract_datetime']]
    df_record_types = df_records_raw[['record_id','types', 'extract_datetime']]
    df_record_identifiers = df_records_raw[['record_id','identifiers', 'extract_datetime']]
    df_record_languages = df_records_raw[['record_id','languages', 'extract_datetime']]
    df_record_subjects = df_records_raw[['record_id','subjects', 'extract_datetime']]
    df_record_publishers = df_records_raw[['record_id','publishers', 'extract_datetime']]
    df_record_relations = df_records_raw[['record_id','relations', 'extract_datetime']]
    df_record_rights = df_records_raw[['record_id','rights', 'extract_datetime']]
    df_record_sets = df_records_raw[['record_id','set_id', 'extract_datetime']]

    df_record_creators = df_record_creators.explode('creators')
    df_record_types = df_record_types.explode('types')
    df_record_identifiers = df_record_identifiers.explode('identifiers')
    df_record_languages = df_record_languages.explode('languages')
    df_record_subjects = df_record_subjects.explode('subjects')
    df_record_publishers = df_record_publishers.explode('publishers')
    df_record_relations = df_record_relations.explode('relations')
    df_record_rights = df_record_rights.explode('rights')

    df_records['load_datetime'] =  pd.Timestamp.now(tz="UTC").normalize()
    df_record_creators['load_datetime'] = pd.Timestamp.now(tz="UTC").normalize()
    df_record_types['load_datetime'] = pd.Timestamp.now(tz="UTC").normalize()
    df_record_identifiers['load_datetime'] = pd.Timestamp.now(tz="UTC").normalize()
    df_record_languages['load_datetime'] = pd.Timestamp.now(tz="UTC").normalize()
    df_record_subjects['load_datetime'] = pd.Timestamp.now(tz="UTC").normalize()
    df_record_publishers['load_datetime'] = pd.Timestamp.now(tz="UTC").normalize()
    df_record_relations['load_datetime'] = pd.Timestamp.now(tz="UTC").normalize()
    df_record_rights['load_datetime'] = pd.Timestamp.now(tz="UTC").normalize()


    sets_df = df_record_sets['set_id'].apply(pd.Series)
    sets_df = sets_df.rename(columns=lambda i: f'set_{i}')

    # junta con record_id y (opcional) elimina la columna original
    df_record_sets = pd.concat([df_record_sets[['record_id']], sets_df], axis=1)

    return df_records, df_record_creators, df_record_types, df_record_identifiers, df_record_languages, df_record_subjects, df_record_publishers, df_record_relations, df_record_rights, df_record_sets


In [7]:
df_records, df_record_creators, df_record_types, \
    df_record_identifiers, df_record_languages, df_record_subjects, \
        df_record_publishers, df_record_relations, df_record_rights, \
             df_record_sets = oai_load_records(df_records_raw)

[06/12/25 11:33:32] WARNING  /tmp/ipykernel_39418/2868236217.py:24: SettingWithCopyWarning:         ]8;id=646855;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=42970;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             A value is trying to be set on a copy of a slice from a DataFrame.                    
                             Try using .loc = value instead                                                        
                                                                                                                   
                             See the caveats in the documentation:                                                 
                             https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#                
                             returning-a-view-versus-a-copy                                                        
                               df_records['load_datetime'] =                                                       
                             pd.Timestamp.now(tz="UTC").normalize()                                                
                                                                                                                   

In [8]:
df_records

,record_id,col_id,title,date_issued,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/179477,com_11336_35,Paisajes mesetarios en Patagonia: Tecnología d...,2016-12,2025-12-06 00:00:00+00:00,2025-12-06 00:00:00+00:00
1,oai:ri.conicet.gov.ar:11336/116972,com_11336_219,Distancias entre la ecología y la praxis ambie...,2011,2025-12-06 00:00:00+00:00,2025-12-06 00:00:00+00:00
...,...,...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/144165,com_11336_238,Simultaneous Determination of Human Erythrocyt...,2020-10,2025-12-06 00:00:00+00:00,2025-12-06 00:00:00+00:00
197,oai:ri.conicet.gov.ar:11336/54764,com_11336_345,Two-dimensional geoelectrical modeling using a...,2000-05,2025-12-06 00:00:00+00:00,2025-12-06 00:00:00+00:00


In [ ]:
df_record_creators


,item_id,creators,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/215406,"Dillchneider Loza, Alexandra",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
0,oai:ri.conicet.gov.ar:11336/215406,"Frasier, Ileana",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/7132,"Printista, Alicia Marcela",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
196,oai:ri.conicet.gov.ar:11336/7132,"Luque, Emilio",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00


In [ ]:
df_record_types


,item_id,types,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/215406,info:eu-repo/semantics/article,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
0,oai:ri.conicet.gov.ar:11336/215406,info:ar-repo/semantics/artículo,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/7132,info:ar-repo/semantics/artículo,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
196,oai:ri.conicet.gov.ar:11336/7132,info:eu-repo/semantics/publishedVersion,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00


In [ ]:
df_record_identifiers

,item_id,identifiers,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/215406,http://hdl.handle.net/11336/215406,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
0,oai:ri.conicet.gov.ar:11336/215406,"Dillchneider Loza, Alexandra; Frasier, Ileana;...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/7132,"Tissera, Pablo Cristian; Castro, Alicia; Print...",2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
196,oai:ri.conicet.gov.ar:11336/7132,2251-7545,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00


In [ ]:
df_record_languages

,item_id,languages,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/215406,spa,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
1,oai:ri.conicet.gov.ar:11336/140529,eng,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
...,...,...,...,...
195,oai:ri.conicet.gov.ar:11336/235556,eng,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
196,oai:ri.conicet.gov.ar:11336/7132,eng,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00


In [ ]:
df_record_subjects


,item_id,subjects,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/215406,TEXTURA DEL SUELO,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
0,oai:ri.conicet.gov.ar:11336/215406,EFICIENCIA DE USO DEL NITRÓGENO,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/7132,https://purl.org/becyt/ford/1.2,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
196,oai:ri.conicet.gov.ar:11336/7132,https://purl.org/becyt/ford/1,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00


In [ ]:
df_record_publishers


,item_id,publishers,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/215406,Universidad Nacional de La Pampa. Facultad de ...,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
1,oai:ri.conicet.gov.ar:11336/140529,Elsevier Science,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
...,...,...,...,...
195,oai:ri.conicet.gov.ar:11336/235556,Acharya Nagarjuna Unversity,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
196,oai:ri.conicet.gov.ar:11336/7132,Advanced Academic Publisher,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00


In [ ]:
df_record_relations


,item_id,relations,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/215406,info:eu-repo/semantics/altIdentifier/url/https...,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
1,oai:ri.conicet.gov.ar:11336/140529,info:eu-repo/semantics/altIdentifier/url/https...,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/7132,info:eu-repo/semantics/altIdentifier/arxiv/140...,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
196,oai:ri.conicet.gov.ar:11336/7132,info:eu-repo/semantics/altIdentifier/url/http:...,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00


In [ ]:
df_record_rights

,item_id,rights,extract_datetime,load_datetime
0,oai:ri.conicet.gov.ar:11336/215406,info:eu-repo/semantics/openAccess,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
0,oai:ri.conicet.gov.ar:11336/215406,https://creativecommons.org/licenses/by-nc-sa/...,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/7132,info:eu-repo/semantics/openAccess,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00
196,oai:ri.conicet.gov.ar:11336/7132,https://creativecommons.org/licenses/by/2.5/ar/,2025-11-25 00:00:00+00:00,2025-11-25 00:00:00+00:00


In [9]:
df_record_sets

,record_id,set_0,set_1,set_2,set_3,set_4,set_5,set_6,set_7,set_8,set_9,set_10,set_11,set_12,set_13
0,oai:ri.conicet.gov.ar:11336/179477,com_11336_35,com_11336_14,col_11336_36,snrd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,oai:ri.conicet.gov.ar:11336/116972,com_11336_219,com_11336_218,col_11336_89919,snrd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,oai:ri.conicet.gov.ar:11336/144165,com_11336_238,com_11336_231,col_11336_239,snrd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,oai:ri.conicet.gov.ar:11336/54764,com_11336_345,com_11336_332,col_11336_346,snrd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
